In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

# 2.3. The gears of neural networks: tensor operations

- All transformations learned by deep neural networks can be reduced to a handful of **tensor operations** applied to tensors of numeric data. 
- For instance, it’s possible to add tensors, multiply tensors, and so on.


In our initial example, we were building our network by **stacking Dense layers** on top of each other. 

A Keras layer instance looks like this:

keras.layers.Dense(512, activation='relu')

- This layer can be interpreted as a function, which takes as input a 2D tensor and returns another 2D tensor—a new representation for the input tensor.

- Specifically, the function is as follows (where W is a 2D tensor and b is a vector, both attributes of the layer):

output = relu(dot(W, input) + b)

Let’s unpack this. 

**We have three tensor operations here:** 
- a dot product (dot) between the input tensor and a tensor named W; 
- an addition (+) between the resulting 2D tensor and a vector b; 
- and, finally, a relu operation. relu(x) is max(x, 0).

## 2.3.1. Element-wise operations

- The relu operation and addition are element-wise operations: 
- That is, operations that are applied independently to each entry in the tensors being considered. 
- This means these operations are highly amenable to massively parallel implementations (vectorized implementations, a term that comes from the vector processor supercomputer architecture from the 1970–1990 period). 

- If you want to write a naive Python implementation of an element-wise operation, you use a for loop, as in this naive implementation of an element-wise relu operation:

In [2]:
def naive_relu(x):
    assert len(x.shape) == 2                     #  x is a 2D Numpy tensor.

    x = x.copy()                                 #  Avoid overwriting the input tensor.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

You do the same for addition:

In [3]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape                # x and y are 2D Numpy tensors.

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

On the same principle, you can do element-wise multiplication, subtraction, and so on.

- In practice, when dealing with Numpy arrays, these operations are available as well-optimized **built-in Numpy functions**

- So, in Numpy, you can do the following element-wise operation, and it will be blazing fast:

In [4]:
import numpy as np

z = x + y                       # Element-wise Addition

z = np.maximum(z, 0.)           # Element-wise relu

NameError: name 'x' is not defined

## 2.3.2. Broadcasting

- Our earlier naive implementation of naive_add only supports the addition of **2D tensors with identical shapes**. 
- But in the **Dense layer** introduced earlier, **we added a 2D tensor with a vector**. 

### What happens with addition when the shapes of the two tensors being added differ?

When possible, and if there’s no ambiguity, 

- the smaller tensor will be broadcasted to match the shape of the larger tensor. 

### Broadcasting consists of two steps:

1. Axes (called broadcast axes) are added to the smaller tensor to match the ndim of the larger tensor.
1. The smaller tensor is repeated alongside these new axes to match the full shape of the larger tensor.

### Let’s look at a concrete example. 

- Consider X with shape (32, 10) and y with shape (10,). 
- First, we add an empty first axis to y, whose shape becomes (1, 10). 
- Then, we repeat y 32 times alongside this new axis, so that we end up with a tensor Y with shape (32, 10), where Y[i, :] == y for i in range(0, 32). 
- At this point, we can proceed to add X and Y, because they have the same shape.

- In terms of implementation, no new 2D tensor is created, because that would be terribly inefficient. 
- The repetition operation is entirely virtual: **it happens at the algorithmic level rather than at the memory level**. 
- But thinking of the vector being repeated 10 times alongside a new axis is a helpful mental model. 

### Here’s what a naive implementation would look like:

In [ ]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2                           #  x is a 2D Numpy tensor
    assert len(y.shape) == 1                           #  y is a Numpy vector.
    assert x.shape[1] == y.shape[0]

    x = x.copy()                                       #  Avoid overwriting the input tensor
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

- With broadcasting, you can generally apply two-tensor element-wise operations if one tensor has shape (a, b, ... n, n + 1, ... m) and the other has shape (n, n + 1, ... m). 

- The broadcasting will then automatically happen for axes a through n - 1.

### The following example applies the element-wise maximum operation to two tensors of different shapes via broadcasting:

In [ ]:
import numpy as np

x = np.random.random((64, 3, 32, 10))        # x is a random tensor with shape (64, 3, 32, 10)
y = np.random.random((32, 10))              #  y is a random tensor with shape (32, 10).

z = np.maximum(x, y)                        # The output z has shape (64, 3, 32, 10) like x

## 2.3.3. Tensor dot

- The **dot operation**, also called a **tensor product** (not to be confused with an element-wise product) is the most common, most useful tensor operation. 
- Contrary to element-wise operations, it combines entries in the input tensors.

- An element-wise product is done with the * operator in Numpy, Keras, Theano, and TensorFlow. 
- dot uses a different syntax in TensorFlow, but in both Numpy and Keras it’s done using the standard dot operator:

import numpy as np
z = np.dot(x, y)

from keras import backend as K
z = K.dot(x, y)

In mathematical notation, you’d note the operation with a dot (.):

z = x . y

Mathematically, what does the dot operation do? 
Let’s start with the dot product of two vectors x and y. 

It’s computed as follows:

In [ ]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]

    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z
     
    # x and y are Numpy vectors.

- The dot product between two vectors is a **scalar**. 
- Only vectors with **the same number** of elements are compatible for a dot product.

You can also take **the dot product between a matrix x and a vector y**, which returns **a vector where the coefficients are the dot products between y and the rows of x**. 


You implement it as follows:

In [ ]:
import numpy as np

def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]

    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

You could also reuse the code we wrote previously, which highlights the relationship between a matrix-vector product and a vector product:

In [ ]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

Note that as soon as one of the two tensors has an ndim greater than 1, 

dot is no longer symmetric (commutative), which is to say that **dot(x, y) isn’t the same as dot(y, x)**.

- Of course, a dot product generalizes to tensors with an arbitrary number of axes. 
- The most common applications may be the dot product between two matrices. 
- You can take the dot product of two matrices x and y (dot(x, y)) if and only if x.shape[1] == y.shape[0]. 
- The result is a matrix with shape (x.shape[0], y.shape[1]), where the coefficients are the vector products between the rows of x and the columns of y. 

Here’s the naive implementation:

In [ ]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]

    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

To understand dot-product shape compatibility, it helps to visualize the input and output tensors by aligning them as shown in figure 2.5.

<img src = "images/Fig2-5.png">

- x, y, and z are pictured as rectangles (literal boxes of coefficients). 
- Because the rows x and the columns of y must have the same size, it follows that the width of x must match the height of y.

More generally, you can take the dot product between higher-dimensional tensors, following the same rules for shape compatibility as outlined earlier for the 2D case:

(a, b, c, d) . (d,) -> (a, b, c)

(a, b, c, d) . (d, e) -> (a, b, c, e)

## 2.3.4. Tensor reshaping

-  It is essential to understand is **tensor reshaping**. 
- Although it wasn’t used in the Dense layers in our first neural network example, we used it when we preprocessed the digits data before feeding it into our network:

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
train_images.shape

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))

In [ ]:
train_images.shape

- **Reshaping a tensor means rearranging its rows and columns to match a target shape**. 
- Naturally, the reshaped tensor has the same total number of coefficients as the initial tensor. 

Reshaping is best understood via simple examples:

In [ ]:
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])
print(x.shape)

In [ ]:
x = x.reshape((6, 1))
x

In [ ]:
x.shape

In [ ]:
x = x.reshape((2, 3))
x

In [ ]:
x.shape

- A special case of reshaping that’s commonly encountered is **transposition**. 
- Transposing a matrix means **exchanging its rows and its columns**, so that x[i, :] becomes x[:, i]:

In [ ]:
x = np.zeros((300, 20))

x = np.transpose(x)

print(x.shape)

## 2.3.5. Geometric interpretation of tensor operations

- Because the contents of the tensors manipulated by tensor operations can be interpreted as coordinates of points in some geometric space, **all tensor operations have a geometric interpretation**. 
- For instance, let’s consider addition. We’ll start with the following vector:

In [ ]:
A = [0.5, 1]

It’s a point in a 2D space (see figure 2.6). It’s common to picture a vector as an arrow linking the origin to the point, as shown in figure 2.7.

<img src = "images/Fig2-6.png">

<img src = "images/Fig2-7.png">

- Let’s consider a new point, B = [1, 0.25], which we’ll add to the previous one. 
- This is done geometrically by chaining together the vector arrows, with the resulting location being the vector representing the sum of the previous two vectors (see figure 2.8).

<img src = "images/Fig2-8.png">

- In general, **elementary geometric operations such as affine transformations, rotations, scaling, and so on can be expressed as tensor operations.** 
- For instance, a rotation of a 2D vector by an angle theta can be achieved via a dot product with a 2 × 2 matrix R = [u, v], where u and v are both vectors of the plane: u = [cos(theta), sin(theta)] and v = [-sin(theta), cos(theta)].

## 2.3.6. A geometric interpretation of deep learning

You just learned that 

- **Neural networks consist entirely of chains of tensor operations**.
- All of these tensor operations are just **geometric transformations of the input data**. 
- It follows that you can interpret a neural network as **a very complex geometric transformation in a high-dimensional space**, implemented via a long series of simple steps.

In 3D, the following mental image may prove useful. 

- Imagine two sheets of colored paper: one red and one blue. 
- Put one on top of the other. 
- Now crumple them together into a small ball. 
- That crumpled paper ball is your input data, and each sheet of paper is a class of data in a classification problem. 
- What a neural network (or any other machine-learning model) is meant to do is figure out a transformation of the paper ball that would uncrumple it, so as to make the two classes cleanly separable again. 
- With deep learning, this would be implemented as a series of simple transformations of the 3D space, such as those you could apply on the paper ball with your fingers, one movement at a time.

Figure 2.9. Uncrumpling a complicated manifold of data
<img src = "images/Fig2-9.png">

- Uncrumpling paper balls is what machine learning is about: **finding neat representations for complex, highly folded data manifolds**. 
- At this point, you should have a pretty good intuition as to why deep learning excels at this: it takes the approach of incrementally decomposing a complicated geometric transformation into a long chain of elementary ones, which is pretty much the strategy a human would follow to uncrumple a paper ball. 
- Each layer in a deep network applies a transformation that disentangles the data a little—and a deep stack of layers makes tractable an extremely complicated disentanglement process.